In [2]:
pip install numpy

     |████████████████████████████████| 16.0 MB 2.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
     |████████████████████████████████| 2.9 MB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 253 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 30.8 MB 10.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

     |████████████████████████████████| 173.9 MB 2.6 kB/s eta 0:00:011  |█▍                              | 7.5 MB 6.8 MB/s eta 0:00:25     |██████████████████              | 97.3 MB 5.2 MB/s eta 0:00:15
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 1.0 MB 6.1 MB/s eta 0:00:01
  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
     |████████████████████████████████| 3.0 MB 11.5 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorboard-2.4.1-py3-none-any.whl (10.6 MB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
     |████████████

In [7]:
# Import Libraries
import numpy as np
import json

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [2]:
#Upload vgg16
vgg16=VGG16(weights='imagenet')

In [3]:
vgg16.save('vgg16.h5')

In [10]:
def prepare_image(img_path):
    img=load_img(img_path,target_size=(224,224)) #returns A PIL Image instance.
    x=img_to_array(img)#returns 3D numpy array
    x=np.expand_dims(x,axis=0) #https://stackoverflow.com/questions/47555829/preprocess-input-method-in-keras/47556342 -- we need extra dimension for number of samples as keras processes in batch
    x=preprocess_input(x) #image_data_format is channel_last by default
    # https://github.com/tensorflow/tensorflow/blob/85c8b2a817f95a3e979ecd1ed95bff1dc1335cff/tensorflow/python/keras/applications/imagenet_utils.py#L115
    # @keras_export('keras.applications.imagenet_utils.preprocess_input') def _preprocess_numpy_input(x, data_format, mode)
    
    #the last part uses the 'caffe' default mode i.e.converts rgb to bgr and centres the data for every channel around 0 (dosnt scale)
    return x

In [74]:
pip install pillow # need to install for PIL

     |████████████████████████████████| 2.2 MB 1.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
y = prepare_image('/Users/rishitdholakia/Desktop/car.jpg')

In [13]:
y.shape

(1, 224, 224, 3)

In [15]:
preds=vgg16.predict(y)

In [16]:
preds

array([[1.95381222e-08, 1.88057978e-08, 2.57546322e-08, 3.90971877e-09,
        4.16368664e-08, 3.81560668e-08, 5.30795097e-09, 4.30993578e-06,
        1.35602886e-06, 4.79385029e-08, 2.96553342e-08, 5.10813010e-08,
        1.14171058e-07, 9.22943677e-09, 3.30590098e-08, 1.67561993e-07,
        1.26675701e-07, 9.96689806e-08, 1.31174431e-07, 2.41219880e-08,
        1.81607493e-08, 2.71857033e-07, 4.47500639e-07, 2.40433951e-06,
        1.59872599e-07, 4.63498736e-07, 5.25304813e-08, 4.22753743e-08,
        3.76304143e-08, 5.34334088e-09, 3.61730315e-07, 4.52036119e-07,
        1.28863405e-06, 3.85863103e-07, 1.18645607e-06, 4.39275226e-07,
        3.16868523e-07, 2.53712142e-06, 1.14008820e-07, 8.10200333e-08,
        6.36227469e-07, 2.17441453e-07, 2.67097647e-07, 4.83078573e-08,
        1.71058758e-07, 8.40208401e-08, 3.01453412e-07, 3.40953399e-08,
        1.44401056e-07, 4.23960920e-08, 4.19323065e-08, 8.94106336e-08,
        1.60079026e-07, 3.90550277e-08, 5.37219762e-08, 2.307420

In [17]:
preds.shape

(1, 1000)

In [106]:
CLASS_INDEX= None

In [107]:
CLASS_INDEX_PATH = '/Users/rishitdholakia/Desktop/Car Damage Detection project/imagenet_class_index.json'
#CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

In [108]:
def get_predictions(preds,top=5):
    global CLASS_INDEX #to make changes to global class_index
    CLASS_INDEX=json.load(open(CLASS_INDEX_PATH)) #converts JSON object to python Dictionary
    results=[]
    
    #indices=preds.argsort(axis=-1)[:,-5][:,::-1] #argsort returns indices sorted according to specified axis -- if we dont use reshape
    
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result=[tuple(CLASS_INDEX[str(i)])+(pred[i],) for i in top_indices] # Empty tuples will appear as coral = (), but tuples with even one value must use a comma as in coral = ('blue coral',)
        results.append(result)
    return results

In [116]:
print (get_predictions(preds, top=5))

<class 'list'>


In [110]:
#First Check - category list creation

In [111]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [112]:
 img_list=os.listdir('/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/validation/01-whole')#This method returns a list containing the names of the entries in the directory given by path.
 print(img_list)
 for i,img_path in enumerate(img_list):
     img = prepare_image('/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/validation/01-whole/'+img_path)
 print(img)

['0071.jpg', '0065.jpg', '0059.jpg', '0105.jpg', '0111.jpg', '0139.jpg', '0138.jpg', '0110.jpg', '0104.jpg', '0058.jpg', '0064.jpg', '0070.jpg', '0066.jpg', '0072.jpg', '0099.jpg', '0112.jpg', '0106.jpg', '0107.jpg', '0113.jpg', '0098.jpg', '0073.jpg', '0067.jpg', '0063.jpg', '0077.jpg', '0088.jpg', '0117.jpg', '0103.jpg', '0102.jpg', '0116.jpg', '0089.jpg', '0076.jpg', '0062.jpg', '0048.jpg', '0074.jpg', '0060.jpg', '0128.jpg', '0100.jpg', '0114.jpg', '0115.jpg', '0101.jpg', '0129.jpg', '0061.jpg', '0075.jpg', '0049.jpg', '0012.jpg', '0006.jpg', '0210.jpg', '0204.jpg', '0166.jpg', '0172.jpg', '0199.jpg', '0198.jpg', '0173.jpg', '0167.jpg', '0205.jpg', '0211.jpg', '0007.jpg', '0013.jpg', '0005.jpg', '0011.jpg', '0039.jpg', '0207.jpg', '0213.jpg', '0171.jpg', '0165.jpg', '0159.jpg', '0158.jpg', '0164.jpg', '0170.jpg', '0212.jpg', '0206.jpg', '0038.jpg', '0010.jpg', '0004.jpg', '0028.jpg', '0014.jpg', '0202.jpg', '0216.jpg', '0148.jpg', '0174.jpg', '0160.jpg', '0161.jpg', '0175.jpg', '01

In [126]:
def get_car_categories():
    d=defaultdict(float) #Usually, a Python dictionary throws a KeyError if you try to get an item with a key that is not currently in the dictionary. The defaultdict in contrast will simply create any items that you try to access (provided of course they do not exist yet).
    img_list=os.listdir('/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/validation/01-whole')#This method returns a list containing the names of the entries in the directory given by path.
    for i,img_path in enumerate(img_list):
        img = prepare_image('/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/validation/01-whole/'+img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:  #iterating through tuple using a for loop
            d[j[0:2]]+=j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [127]:
cat_counter=get_car_categories() 

0 / 230 complete
100 / 230 complete
200 / 230 complete


In [128]:
cat_counter

Counter({('n04285008', 'sports_car'): 28.853904586285353,
         ('n04037443', 'racer'): 7.548823233228177,
         ('n03100240', 'convertible'): 22.27065716101788,
         ('n02974003', 'car_wheel'): 5.018784584943205,
         ('n02814533', 'beach_wagon'): 21.272268172120675,
         ('n03930630', 'pickup'): 17.554288666695356,
         ('n02930766', 'cab'): 6.648977704113349,
         ('n03710193', 'mailbox'): 0.11574728786945343,
         ('n03425413', 'gas_pump'): 0.09956557303667068,
         ('n03902125', 'pay-phone'): 0.07892029732465744,
         ('n02791270', 'barbershop'): 0.06867709010839462,
         ('n04461696', 'tow_truck'): 4.083389501320198,
         ('n03977966', 'police_van'): 2.266874913824722,
         ('n03770679', 'minivan'): 35.39320989861153,
         ('n03670208', 'limousine'): 12.064467553049326,
         ('n03445924', 'golfcart'): 0.8001158833503723,
         ('n03459775', 'grille'): 4.1132087131845765,
         ('n03769881', 'minibus'): 3.650426773121

In [129]:
#with open('cat_counter.pk', 'wb') as f:
#    pk.dump(cat_counter,f,-1)              #storing dictionary

In [130]:
#This piece of code will be used to load already saved cat_counter
with open('cat_counter.pk', 'rb') as f:
     cat_counter = pk.load(f)  

In [138]:
cat_list  = [k for k,v in cat_counter.most_common()[:23]]

In [139]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n02930766', 'cab'),
 ('n03594945', 'jeep'),
 ('n02974003', 'car_wheel'),
 ('n03459775', 'grille'),
 ('n04461696', 'tow_truck'),
 ('n03769881', 'minibus'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04065272', 'recreational_vehicle'),
 ('n02701002', 'ambulance'),
 ('n04252225', 'snowplow'),
 ('n03777568', 'Model_T'),
 ('n03445924', 'golfcart'),
 ('n04467665', 'trailer_truck'),
 ('n03417042', 'garbage_truck'),
 ('n03478589', 'half_track')]

# First Check - car or not

In [141]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print ("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list: #searching in the category list -- use in
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [142]:
car_categories_gate('/Users/rishitdholakia/Desktop/car.jpg', cat_list)

Validating that this is a picture of your car...
('n03930630', 'pickup')


'Validation complete - proceed to damage evaluation'

In [143]:
car_categories_gate('/Users/rishitdholakia/Desktop/straight kitchen link in university city.jpg', cat_list)

Validating that this is a picture of your car...


'Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again.'